# <center> 8. Закрепление знаний

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела (вес в кг/(рост в м)^2).

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

In [120]:
ddf=diabetes.copy()
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


Задание 8.1

In [121]:
ddf = ddf.drop_duplicates()
ddf.shape[0]

768

Задание 8.2


In [122]:
def delete_low_information_cols(data_frame, percent):
    low_information_cols = []
    for col in data_frame.columns:
            top_freq = data_frame[col].value_counts(normalize=True).max()
            nunique_ratio = data_frame[col].nunique() / data_frame[col].count()
            if top_freq > percent / 100:
                low_information_cols.append(col)
                print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
            if nunique_ratio > percent / 100:
                low_information_cols.append(col)
                print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
    cleaned_data_frame = data_frame.drop(low_information_cols, axis=1)
    return cleaned_data_frame, low_information_cols

In [123]:
ddf, low_information_cols = delete_low_information_cols(ddf, 95)
low_information_cols

Gender: 100.0% одинаковых значений


['Gender']

Задание 8.3

In [124]:
zero_cols_list = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
ddf[ddf[zero_cols_list]==0] = np.nan
round(ddf['Insulin'].isnull().mean(), 2)

0.49

Задание 8.4

In [125]:
ddf = ddf.dropna(thresh=ddf.shape[0]*0.7, axis=1)
ddf.shape[1]

8

Задание 8.5

In [126]:
ddf = ddf.dropna(thresh=ddf.shape[1]-2, axis=0)
ddf.shape[0]

761

Задание 8.6

In [129]:
values = {}
for col in ddf.columns:
    if ddf[col].isnull().sum() > 0:
        values[col] = ddf[col].median()
ddf = ddf.fillna(values)
round(ddf['SkinThickness'].mean(), 1)

29.1

Задание 8.7

In [130]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [137]:
outliers, cleaned = outliers_iqr(ddf, 'SkinThickness')
outliers.shape[0]

87

Задание 8.8

In [139]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [140]:
outliers, cleaned = outliers_z_score(ddf, 'SkinThickness')
outliers.shape[0]

4

Задание 8.9

In [141]:
def outliers_iqr_mod_log(data, feature, log_scale=False, left=1.5, right=1.5):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [143]:
outliers, cleaned = outliers_iqr_mod_log(ddf, 'DiabetesPedigreeFunction')
outliers_log, cleaned_log = outliers_iqr_mod_log(ddf, 'DiabetesPedigreeFunction', True)
outliers.shape[0] - outliers_log.shape[0]

29